In [64]:
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_auc_score

In [65]:
ORDER = ['conflicto', 'economico', 'humanidad', 'moral']

N = 2

def process_folds(datasets):
    final_dataset = []
    for dataset in datasets:
        data = {}
        def map_array(x):
            string = " "
            if "," in x:
                string = ","
            return [int(i) for i in x.strip("()").strip("[]").split(string)]
        
        def map_y_pred_prob(x):
            return [float(i) for i in x.strip("[]").split(",")]

        y_true = dataset['y_true'].tolist()
        y_pred = dataset['y_pred'].tolist()
        y_pred_prob = dataset['y_prob'].tolist()
        for index, frame in enumerate(ORDER):

            y_true_frame = [x[index] for x in y_true]
            y_pred_frame = [x[index] for x in y_pred]
            y_pred_prob_frame = [x[index] for x in y_pred_prob]
            report = classification_report(y_true_frame, y_pred_frame, output_dict=True, zero_division=False)

            for class_ in ["0", "1"]:
                for key in report[class_]:
                    data[f"{frame}_{class_}_{key}"]=float(report[class_][key])

            for measure in ["macro avg", "weighted avg"]:
                for key in report[measure]:
                    data[f"{frame}_{measure}_{key}"]=float(report[measure][key])
                    
            data[f"{frame}_acc"] = float(report["accuracy"])

            precision, recall, _ = precision_recall_curve(y_true_frame, y_pred_prob_frame)

            data[f"{frame}_auc_press_recall"] = auc(recall, precision)
            data[f"{frame}_roc_auc_score"] = roc_auc_score(y_true_frame, y_pred_prob_frame)

        final_dataset.append(data)

    return pd.DataFrame.from_dict(final_dataset)

def build_report(data, data_std, model, embedding):
    all_recall_macro = []
    all_auc = []
    all_roc = []
    
    all_information = []

    for frame in ['conflicto', 'economico', 'humanidad', 'moral']:
        recall_macro = data[f'{frame}_macro avg_recall'][0]
        acc = data[f'{frame}_acc'][0]
        auc = data[f'{frame}_auc_press_recall'][0]
        roc = data[f'{frame}_roc_auc_score'][0]
        all_information.append([model, embedding, frame, acc, recall_macro, auc, roc])

    return all_information

# Resultados dataset

In [66]:
import os

models = ["SVM", "SVM_Linear", "RF", "MLP-1", "MLP-LR",  "random", "naive_bayes", ]
embeddings = ["beto_embedding_cls", "beto_embedding_mean", "elmo", "tf-idf", "fasttext", "baseline"]

all_information = []
for model in models:
    for emb in embeddings:
        name = "cross_validation#{}#{}.pickle".format(model, emb)
        if not os.path.exists("Results/" + name):
            continue
            
        with open(f"Results/{name}", "rb") as file:
            datasets = pickle.load(file)
      
        print(name)
        data = process_folds(datasets)
        aux = build_report(pd.DataFrame(data.mean()).T, data.applymap(lambda x:0), model, emb)
        all_information.extend(aux)

cross_validation#SVM#beto_embedding_cls.pickle
cross_validation#SVM#beto_embedding_mean.pickle
cross_validation#SVM#elmo.pickle
cross_validation#SVM#tf-idf.pickle
cross_validation#SVM#fasttext.pickle
cross_validation#SVM_Linear#beto_embedding_cls.pickle
cross_validation#SVM_Linear#beto_embedding_mean.pickle
cross_validation#SVM_Linear#elmo.pickle
cross_validation#SVM_Linear#tf-idf.pickle
cross_validation#SVM_Linear#fasttext.pickle
cross_validation#RF#beto_embedding_cls.pickle
cross_validation#RF#beto_embedding_mean.pickle
cross_validation#RF#elmo.pickle
cross_validation#RF#tf-idf.pickle
cross_validation#RF#fasttext.pickle
cross_validation#MLP-1#beto_embedding_cls.pickle
cross_validation#MLP-1#beto_embedding_mean.pickle
cross_validation#MLP-1#elmo.pickle
cross_validation#MLP-1#tf-idf.pickle
cross_validation#MLP-1#fasttext.pickle
cross_validation#MLP-LR#beto_embedding_cls.pickle
cross_validation#MLP-LR#beto_embedding_mean.pickle
cross_validation#MLP-LR#elmo.pickle
cross_validation#MLP-LR

In [67]:
otros = [
    ["cross_validation_Bi-LSTM.pickle", "BILSTM", "End_to_End"],
    ["cross_validation_Bi-LSTM_AsymetricLoss.pickle", "BILSTM AsymetricLoss", "End_to_End"],
    ["cross_validation_Beto-finetunning_cross_entropy.pickle", "Beto Finetunning", "End_to_End"],
    ["cross_validation_Beto-finetunning_asymetric.pickle", "Beto Finetunning AsymetricLoss", "End_to_End"],
]

for name, model, embedding in otros:
    with open(f"Results/{name}", "rb") as file:
        datasets = pickle.load(file)

    data = process_folds(datasets)
    aux = build_report(pd.DataFrame(data.mean()).T, data.applymap(lambda x:0), model, embedding)
    all_information.extend(aux)


In [68]:
set([x[0] for x in all_information])

{'BILSTM',
 'BILSTM AsymetricLoss',
 'Beto Finetunning',
 'Beto Finetunning AsymetricLoss',
 'MLP-1',
 'MLP-LR',
 'RF',
 'SVM',
 'SVM_Linear',
 'naive_bayes',
 'random'}

In [70]:
df = pd.DataFrame(all_information, columns=["Model", "Embedding", "Frame", "ACC", "Recall Macro", "AUC P/R", "ROC"])
df = df.round(2)
df = df[["Model", "Embedding", "Frame", "ACC", "ROC"]]

def highlight_max(s):
    is_max = (s == s.max())
    if s.dtype == "float64":
        return ['background-color: red' if v else '' for v in is_max]
    return ['' for v in is_max]

pivoted = df.pivot_table(index = ["Model", "Embedding"],
                         columns = ['Frame'],

                         margins = False, # total column
                         margins_name = 'All frame').sort_index()

df2 = pivoted.swaplevel(axis=1).sort_index(axis=1, level="Frame").swaplevel(axis=0).sort_index().style.apply(highlight_max).set_precision(2)
df2

In [40]:
pivoted = df.pivot_table(index = ["Model", "Embedding"],
                         columns = ['Frame'],
                         margins = False, # total column
                         margins_name = 'All frame').sort_index()

In [41]:
df1 = pivoted.swaplevel(axis=1).sort_index(axis=1, level="Frame").sort_index()
df1.style.apply(highlight_max).set_precision(2)

In [203]:
df2 = pivoted.swaplevel(axis=1).sort_index(axis=1, level="Frame").swaplevel(axis=0).sort_index()
df2.style.apply(highlight_max).set_precision(2)

In [204]:
df3 = df.groupby(["Embedding", "Model"]).mean()
df3.style.apply(highlight_max).set_precision(N)

In [205]:
df4 = df.groupby(["Embedding", "Model"]).mean().swaplevel(axis=0).sort_index()
df4.style.apply(highlight_max).set_precision(N)